In [7]:
import sys
import os
from pathlib import Path
import time
import json
import math

# Add source directories to Python path
current_dir = Path.cwd()
project_root = current_dir.parent  # Go up one level from notebooks to project root
src_dir = project_root / "src"

# Add paths
sys.path.append(str(src_dir / "data_pipline"))
sys.path.append(str(src_dir / "EU_XML_data_loader"))


from data_pipline import DataPipeline
import get_raw_data_paths_EPO  
from xml_loader_EPO import process_xml_files_list


# Test the new JSON loading functionality
sys.path.append(str(src_dir / "data_pipline" / "json_loader"))

# Import the JSON loader functions
from json_loader_epo import get_epo_json_file_paths, get_all_json_file_paths, load_json_documents

print("🔍 Testing JSON File Loading Functions")
print("=" * 50)

print(f"📁 Current directory: {current_dir}")
print(f"📁 Project root: {project_root}")
print(f"📁 Source directory: {src_dir}")
print(f"✅ Python paths configured")

🔍 Testing JSON File Loading Functions
📁 Current directory: /app/notebooks
📁 Project root: /app
📁 Source directory: /app/src
✅ Python paths configured


In [8]:
import json
import glob
import os
from langchain.docstore.document import Document
import sys
# Prepare .json file for embedding
def extract_documents(json_data):
    bibliographic = json_data.get("bibliographic_data", {})
    doc_id = bibliographic.get("doc_id", "UNKNOWN")
    documents = []

    # Common metadata to propagate
    common_meta = {
        "doc_id": doc_id,
        "language": bibliographic.get("language"),
        "country": bibliographic.get("country"),
        "doc_number": bibliographic.get("doc_number"),
        "application_number": bibliographic.get("application_number"),
        "publication_date": bibliographic.get("publication_date"),
        "ipc_classes": bibliographic.get("ipc_classes", []),
        "file":bibliographic.get("file")
    }

    # Title (en preferred)
    title_dict = bibliographic.get("title", {})
    title = title_dict.get("en") or next(iter(title_dict.values()), "")
    if title:
        documents.append(Document(
            page_content=title,
            metadata={**common_meta, "section": "title"}
        ))

    # Abstract
    abstract = bibliographic.get("abstract")
    if abstract:
        documents.append(Document(
            page_content=abstract,
            metadata={**common_meta, "section": "abstract"}
        ))

    # Claims
    for claim in json_data.get("claims", []):
        documents.append(Document(
            page_content=claim["text"],
            metadata={**common_meta, "section": "claim", "claim_number": claim.get("claim_number")}
        ))

    # Main sections
    for section in json_data.get("main_sections", []):
        section_name = section.get("heading_text", "UNKNOWN_SECTION")
        for p in section.get("paragraphs", []):
            documents.append(Document(
                page_content=f"{section_name}\n{p['text']}",
                metadata={**common_meta, "section": section_name, "p_id": p.get("p_id")}
            ))

    return documents


In [53]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np

# Load tokenizer and embedding model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def count_tokens(text):
    return len(tokenizer.encode(text, add_special_tokens=False))

def get_chunk_size(text: str, total_tokens: int, base_chunk_size: int = 350, min_chunk_size: int = 180) -> int:
    if total_tokens <= base_chunk_size:
        return total_tokens

    num_splits = total_tokens // base_chunk_size
    if total_tokens % base_chunk_size != 0:
        num_splits += 1
    
    balanced_chunk_size = math.ceil(total_tokens / num_splits)

    balanced_chunk_size = int(max(min_chunk_size, balanced_chunk_size))
    return balanced_chunk_size

def chunk_text_by_tokens(text: str, chunk_size: int, overlap : int = 30):
    print("Starting tokenization...")
    input_ids = tokenizer.encode(text, add_special_tokens=False)
    print(f"Tokenized text length: {len(input_ids)} tokens")
    
    # Safety check to prevent infinite loops or extremely slow processing
    # if chunk_size <= 350:
    #     overlap = 0
    
    chunks = []
    start = 0
    
    # Process in batches for large texts
    while start < len(input_ids):
        print(f"Processing chunk at position {start}/{len(input_ids)}")
        end = min(start + chunk_size, len(input_ids))
        chunk_ids = input_ids[start:end]
        
        try:
            chunk_text = tokenizer.decode(chunk_ids)
            chunks.append(chunk_text)
        except Exception as e:
            print(f"Error decoding chunk: {e}")
            # Skip this chunk or use a fallback approach
        print("OVERLAP = ", overlap)
        # Advance positionΩ
        start += chunk_size - overlap
        
        # Safety check to prevent memory issues with too many chunks
        if len(chunks) > 1000:  # Arbitrary limit
            print("⚠️ Warning: Reached maximum number of chunks")
            break
    
    print(f"Created {len(chunks)} chunks")
    return chunks

def embed_document(text: str):
    total_tokens = count_tokens(text)
    chunk_size = get_chunk_size(text, total_tokens)

    chunks = chunk_text_by_tokens(text, chunk_size)
    embeddings = embedding_model.encode(chunks)

    return list(zip(chunks, embeddings))  # list of (chunk, embedding_vector)


2025-06-18 12:44:31,439 - INFO - Use pytorch device_name: cpu
2025-06-18 12:44:31,440 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [57]:
import time
from typing import List, Dict, Any
import logging
import numpy as np
from langchain.schema import Document

def embed_documents_with_metadata(documents, debug=True):
    """Process a list of Document objects, preserving metadata"""
    if debug:
        print(f"📋 Starting document embedding process for {len(documents)} documents")
        start_time = time.time()
    
    processed_docs = []
    
    # Track statistics for debugging
    total_tokens = 0
    total_chunks = 0
    errors = 0
    
    for doc_idx, doc in enumerate(documents):
        if debug and doc_idx % 10 == 0:  # Progress update every 10 docs
            print(f"⏳ Processing document {doc_idx+1}/{len(documents)} ({(doc_idx+1)/len(documents)*100:.1f}%)")
        
        try:
            # Get text content and metadata
            text = doc.page_content
            metadata = doc.metadata
            
            if debug and doc_idx < 3:  # Show sample of first few documents
                print(f"\n📄 Document {doc_idx+1} Preview:")
                print(f"  - Content: {text[:100]}..." if len(text) > 100 else f"  - Content: {text}")
                print(f"  - Metadata: {metadata}")

            
            # Calculate chunk size
            doc_tokens = count_tokens(text)
            total_tokens += doc_tokens
            chunk_size = get_chunk_size(text, doc_tokens)
            
            if debug and doc_idx < 3:
                print(f"  - Token count: {doc_tokens}")
                print(f"  - Calculated chunk size: {chunk_size}")
            
            # Create chunks
            chunk_start_time = time.time()
            print("x")
            if (doc_tokens < 350) :
                overlap = 0
            else :
                overlap = 32
            chunks = chunk_text_by_tokens(text, chunk_size, overlap)
            print("y")
            total_chunks += len(chunks)
            
            if debug and doc_idx < 3:
                chunk_time = time.time() - chunk_start_time
                print(f"  - Chunks created: {len(chunks)} (in {chunk_time:.2f}s)")
                print(f"  - First chunk: {chunks[0][:50]}...")
            
            # Generate embeddings for all chunks
            embedding_start_time = time.time()
            try:
                
                batch_size = 8 
                all_embeddings = []
                
                for batch_idx in range(0, len(chunks), batch_size):
                    batch = chunks[batch_idx:batch_idx+batch_size]
                    print(f"  - Processing batch {batch_idx//batch_size + 1}/{(len(chunks)+batch_size-1)//batch_size}")
                    batch_embeddings = embedding_model.encode(batch, show_progress_bar=False)
                    all_embeddings.extend(batch_embeddings)
                
                embeddings = all_embeddings
                print("✓ Embedding completed")
            except Exception as e:
                print(f"❌ Error during embedding: {str(e)}")
                raise
            print("x")
            if debug and doc_idx < 3:
                embedding_time = time.time() - embedding_start_time
                print(f"  - Embeddings generated: {len(embeddings)} vectors of shape {embeddings[0].shape}")
                print(f"  - Embedding time: {embedding_time:.2f}s ({embedding_time/len(chunks):.4f}s per chunk)")
            
            # Create new documents with chunks and metadata
            for i, (chunk_text, embedding) in enumerate(zip(chunks, embeddings)):
                # Create a copy of metadata and add chunk information
                chunk_metadata = metadata.copy()
                chunk_metadata["chunk_index"] = i
                chunk_metadata["total_chunks"] = len(chunks)
                chunk_metadata["source_doc_idx"] = doc_idx
                
                # Check for NaN values in embedding
                if np.isnan(embedding).any():
                    if debug:
                        print(f"⚠️ Warning: NaN values detected in embedding for document {doc_idx}, chunk {i}")
                    # Replace NaN with zeros
                    embedding = np.nan_to_num(embedding)
                
                # Create a new Document object with chunk text and original metadata
                processed_docs.append({
                    "text": chunk_text,
                    "embedding": embedding,
                    "metadata": chunk_metadata
                })
                
        except Exception as e:
            errors += 1
            if debug:
                print(f"❌ Error processing document {doc_idx}: {str(e)}")
    
    if debug:
        end_time = time.time()
        total_time = end_time - start_time
        print(f"\n✅ Embedding completed in {total_time:.2f}s")
        print(f"📊 Statistics:")
        print(f"  - Documents processed: {len(documents)}")
        print(f"  - Total tokens: {total_tokens}")
        print(f"  - Total chunks created: {total_chunks}")
        print(f"  - Average chunks per document: {total_chunks/len(documents):.2f}")
        print(f"  - Processing speed: {total_tokens/total_time:.2f} tokens/second")
        print(f"  - Documents with errors: {errors}")
        print(f"  - Success rate: {(len(documents)-errors)/len(documents)*100:.2f}%")
        
        # Memory usage of embeddings
        embedding_size = sum(emb["embedding"].nbytes for emb in processed_docs)
        print(f"  - Embedding memory usage: {embedding_size/1024/1024:.2f} MB")
    
    return processed_docs



In [58]:
# Process all files
file_list = get_epo_json_file_paths()[:10]
all_documents = []

len(file_list)

📁 Found 1286 EPO JSON files


10

In [59]:
for file_path in file_list:
    print(file_path)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            # Extract documents (returns list of Document objects)
            docs = extract_documents(data)
            
            # Process each document with metadata preservation
            chunked_docs = embed_documents_with_metadata(docs)
            all_documents.extend(chunked_docs)
            
        print(f"Processed {file_path}, total documents: {len(all_documents)}")
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

print(f"Total embedded chunks: {len(all_documents)}")

# Example of examining the first few documents
for i, doc in enumerate(all_documents[:3]):
    print(f"\nDocument {i+1}:")
    print(f"Text: {doc['text'][:100]}...")
    print(f"Embedding shape: {doc['embedding'].shape}")
    print(f"Metadata: {doc['metadata']}")

/app/data/parsed/EPO/EPRTBJV2025000024001001/EPW1B9/EP13899497W1B9/EP13899497W1B9.json
📋 Starting document embedding process for 180 documents
⏳ Processing document 1/180 (0.6%)

📄 Document 1 Preview:
  - Content: AUDIO SIGNAL ENCODER
  - Metadata: {'doc_id': 'EP13899497B9W1', 'language': 'en', 'country': 'EP', 'doc_number': '3084761', 'application_number': '13899497.5', 'publication_date': '20250611', 'ipc_classes': ['G10L  19/038       20130101AFI20170426BHEP', 'G10L  19/07        20130101ALI20170426BHEP'], 'file': 'EP13899497W1B9.xml', 'section': 'title'}
  - Token count: 5
  - Calculated chunk size: 5
x
Starting tokenization...
Tokenized text length: 5 tokens
Processing chunk at position 0/5
OVERLAP =  0
Created 1 chunks
y
  - Chunks created: 1 (in 0.00s)
  - First chunk: audio signal encoder...
  - Processing batch 1/1
✓ Embedding completed
x
  - Embeddings generated: 1 vectors of shape (384,)
  - Embedding time: 0.03s (0.0348s per chunk)

📄 Document 2 Preview:
  - Content: A proc

In [14]:
all_documents[3]

{'text': 'squared ; determining the best leader class associated with the single potential code vector which generates the smallest associated distance ; and sorting components of the best leader class by the reverse ordering of the descending order of absolute values of the components of the single vector of parameters to generate an output lattice - quantized vector.',
 'embedding': array([-5.95339723e-02,  5.56145906e-02, -5.51404022e-02, -3.64771783e-02,
        -3.81343700e-02,  4.62850481e-02, -2.59227213e-02, -1.80207603e-02,
        -1.10133621e-03, -8.75405967e-04,  1.88852716e-02,  3.99518460e-02,
         3.90190408e-02, -1.90947379e-03, -9.95191932e-02, -7.86234345e-03,
         1.70263350e-02,  9.65642035e-02, -6.24190159e-02, -4.64931428e-02,
         6.05621003e-02, -7.02885091e-02, -2.86987349e-02,  1.04361326e-02,
         6.36881143e-02, -2.62317099e-02,  4.23041061e-02,  5.52301221e-02,
         9.72821340e-02, -8.66331086e-02,  1.62349530e-02,  2.15308517e-02,
     

In [33]:
all_documents[7]

{'text': 'a plurality of scale factors ; and applying the scale factor to the output lattice - quantized codevector.',
 'embedding': array([ 3.75743280e-03, -2.34160628e-02, -7.15005025e-02, -1.68837663e-02,
        -7.28141982e-03,  5.67938946e-02, -5.36125153e-02, -3.23771387e-02,
        -1.56677645e-02, -2.38026790e-02,  3.40407491e-02, -2.54890732e-02,
        -9.84252244e-03, -1.52092725e-02, -4.42799479e-02,  1.44423805e-02,
        -7.25233257e-02,  8.47656131e-02, -3.66127118e-02,  2.49930541e-04,
         4.34344076e-02, -5.25875725e-02, -6.99369460e-02, -2.12371945e-02,
         9.17417407e-02,  4.19590548e-02, -1.71999000e-02,  3.53301018e-02,
         9.13563818e-02, -2.94876713e-02,  2.89389789e-02,  7.54222944e-02,
         1.14546843e-01,  7.84164220e-02, -2.17469465e-02,  2.62259934e-02,
        -6.37745135e-04, -5.65790571e-02, -7.07795545e-02,  9.23098177e-02,
         1.53841628e-02,  9.80806574e-02,  4.46067713e-02,  1.93364453e-02,
        -8.35358351e-03, -2.9693